# Intermediate arrival times

In [1]:
import pandas as pd

# Data access
from configs.path_manager import get_path

# Utils
from src.utils.initialization import jobs_jssp_init as init, arrivals_init as term

In [2]:
# Load file
basic_data_path = get_path("data", "basic")
df_routings = pd.read_csv(basic_data_path / "ft10_routings.csv")
df_routings

,Routing_ID,Operation,Machine,Processing Time
0,0,0,M00,29
1,0,1,M01,78
2,0,2,M02,9
3,0,3,M03,36
4,0,4,M04,49
...,...,...,...,...
95,9,5,M09,76
96,9,6,M05,47
97,9,7,M03,52
98,9,8,M04,90


## I) Average arrival time t_a

### 1. Vector of processing times on the bottleneck machine (Engpassmaschine)

#### a) Identification of the bottleneck machine (7.12)

In [3]:
engpassmaschine = term._get_engpassmaschine(df_routings, verbose = True)
engpassmaschine

Machine load (total processing time):
  M00: 493
  M01: 548
  M02: 556
  M03: 631
  M04: 534
  M05: 416
  M06: 491
  M07: 499
  M08: 531
  M09: 410


'M03'

#### b) Create vector of processing times on the bottleneck machine

In [4]:
vec_t_b_mmax = term._get_vec_t_b_mmax(df_routings)
vec_t_b_mmax

[36, 69, 39, 98, 26, 95, 61, 79, 76, 52]

### 2. Routings probabilities

In [5]:
# Number of different Routings
n_routings = df_routings['Routing_ID'].nunique()

p = [1.0 / n_routings] * n_routings
p

[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

### 3. Calculate the average inter-arrival time t_a using the formula (7.16)
> All-in-One

In [6]:
t_a = term.calculate_mean_interarrival_time(df_routings, u_b_mmax = 0.9)
t_a

70.11

## II) Arrival times

#### Generation of exponentially distributed inter-arrival times with mean t_a
#### with a random job sequence

In [7]:
df_jssp = init.generate_jssp_from_routings(df_routings, shuffle = True)
df_jobs = df_jssp[["Job", "Routing_ID"]].drop_duplicates()

arrivals = term.generate_arrivals_from_mean_interarrival_time(10, t_a)
df_jobs_arrivals = df_jobs.copy()
df_jobs_arrivals['Arrival'] = arrivals
df_jobs_arrivals

,Job,Routing_ID,Arrival
0,J25-0000,8,0
10,J25-0001,6,50
20,J25-0002,0,118
30,J25-0003,2,164
40,J25-0004,3,207
50,J25-0005,1,418
60,J25-0006,9,711
70,J25-0007,5,846
80,J25-0008,4,909
90,J25-0009,7,925
